# Model- import - test funcions

In [1]:
import os
print(os.environ.get('TF_ENABLE_ONEDNN_OPTS'), os.environ.get('TF_CPP_MIN_LOG_LEVEL'))

import random
from collections import defaultdict
import numpy as np
import tensorflow as tf

import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Dense, AdditiveAttention, Concatenate
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

import string
import matplotlib.pyplot as plt

0 3


E0000 00:00:1749735239.466791 1055125 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749735239.472717 1055125 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# -------------------- Constants --------------------
OPERATORS = ['+', '-', '*', '/']
IDENTIFIERS = list('abcde')
SPECIAL_TOKENS = ['PAD', 'SOS', 'EOS']
SYMBOLS = ['(', ')', '+', '-', '*', '/']
VOCAB = SPECIAL_TOKENS + SYMBOLS + IDENTIFIERS + ['JUNK'] #may use junk in autoregressive generation

token_to_id = {tok: i for i, tok in enumerate(VOCAB)}
id_to_token = {i: tok for tok, i in token_to_id.items()}
VOCAB_SIZE = len(VOCAB)
PAD_ID = token_to_id['PAD']
EOS_ID = token_to_id['EOS']
SOS_ID = token_to_id['SOS']

MAX_DEPTH = 3
MAX_LEN = 4*2**MAX_DEPTH -2 #enough to fit expressions at given depth (+ EOS)

# -------------------- Expression Generation --------------------
def generate_infix_expression(max_depth):
    if max_depth == 0:
        return random.choice(IDENTIFIERS)
    elif random.random() < 0.5:
        return generate_infix_expression(max_depth - 1)
    else:
        left = generate_infix_expression(max_depth - 1)
        right = generate_infix_expression(max_depth - 1)
        op = random.choice(OPERATORS)
        return f'({left} {op} {right})'

def tokenize(expr):
    return [c for c in expr if c in token_to_id]

def infix_to_postfix(tokens):
    precedence = {'+': 1, '-': 1, '*': 2, '/': 2}
    output, stack = [], []
    for token in tokens:
        if token in IDENTIFIERS:
            output.append(token)
        elif token in OPERATORS:
            while stack and stack[-1] in OPERATORS and precedence[stack[-1]] >= precedence[token]:
                output.append(stack.pop())
            stack.append(token)
        elif token == '(':
            stack.append(token)
        elif token == ')':
            while stack and stack[-1] != '(':
                output.append(stack.pop())
            stack.pop()
    while stack:
        output.append(stack.pop())
    return output

def encode(tokens, max_depth=MAX_DEPTH):
    max_len = 4*2**max_depth -2
    ids = [token_to_id[t] for t in tokens] + [EOS_ID]
    return ids + [PAD_ID] * (max_len - len(ids))

def decode_sequence(token_ids, id_to_token, pad_token='PAD', eos_token='EOS'):
    """
    Converts a list of token IDs into a readable string by decoding tokens.
    Stops at the first EOS token if present, and ignores PAD tokens.
    """
    tokens = []
    for token_id in token_ids:
        token = id_to_token.get(token_id, '?')
        if token == eos_token:
            break
        if token != pad_token:
            tokens.append(token)
    return ' '.join(tokens)

def generate_dataset(n,max_depth=MAX_DEPTH):
    X, Y = [], []
    for _ in range(n):
        expr = generate_infix_expression(max_depth)
        infix = tokenize(expr)
        postfix = infix_to_postfix(infix)
        X.append(encode(infix, max_depth = max_depth))
        Y.append(encode(postfix, max_depth= max_depth))
    return np.array(X), np.array(Y)

#you might use the shift function for teacher-forcing
def shift_right(seqs):
    shifted = np.zeros_like(seqs)
    shifted[:, 1:] = seqs[:, :-1]
    shifted[:, 0] = SOS_ID
    return shifted

def prefix_accuracy_single(y_true, y_pred, id_to_token, eos_id=EOS_ID, verbose=False):
    t_str = decode_sequence(y_true, id_to_token).split(' EOS')[0]
    p_str = decode_sequence(y_pred, id_to_token).split(' EOS')[0]
    t_tokens = t_str.strip().split()
    p_tokens = p_str.strip().split()
    max_len = max(len(t_tokens), len(p_tokens))

    match_len = sum(x == y for x, y in zip(t_tokens, p_tokens))
    score = match_len / max_len if max_len>0 else 0

    if verbose:
        print("TARGET :", ' '.join(t_tokens))
        print("PREDICT:", ' '.join(p_tokens))
        print(f"PREFIX MATCH: {match_len}/{len(t_tokens)} → {score:.2f}")

    return score

def autoregressive_decode(encoder_input, max_depth=MAX_DEPTH):
    max_len = 4 * 2**max_depth - 2 
    enc_in = encoder_input[None, :] # Add batch dimension
    enc_outs, h, c = encoder.predict(enc_in, verbose=0)

    token = np.array([[SOS_ID]])
    output_seq = []
    for _ in range(max_len):
        logits, h, c = decoder.predict([token, enc_outs, h, c], verbose=0)
        # greedy decoding
        sampled_id = np.argmax(logits[0, 0, :])
        output_seq.append(sampled_id)
        if sampled_id == EOS_ID:
            break
        token = np.array([[sampled_id]])
    return output_seq

"""
All the adds did to the test function are to make have more or less verbose outputs
"""
def test(no=20,rounds=10,max_depth=MAX_DEPTH,verbose_n=0, verbose_round = True):  # added max depth and verbose
  rscores =[]
  for i in range(rounds):
    if verbose_round:                                                     # added
      print("================================================")           # added
      print(f"round={i}")
      print("------------------------------------------------")           # added
    X_test, Y_test = generate_dataset(no, max_depth)                      # added max depth
    scores = []
    n = verbose_n                                                         # added
    for j in range(no): 
      if n < (no+1):                                                      # added     
        if n <= 0:                                                        # added
          verbose = False                                                 # added
        else:                                                             # added
          verbose = True                                                  # added  
        n -= 1                                                            # added
      encoder_input=X_test[j]
      generated = autoregressive_decode(encoder_input,max_depth=max_depth)                    # [1:] In my case no nedd to remove SOS, the function returns it whitout
      if verbose:                                                         # added
        print(f"~~~~~ es number {j} ~~~~~")                               # added
      scores.append(prefix_accuracy_single(Y_test[j], generated, id_to_token, verbose=verbose))
    if verbose_n > 0:                                                     # added
      print("------------------------------------------------")           # added
    if verbose_round:                                                     # added
      print(f"mean scores={np.mean(scores):.4f} std={np.std(scores):.4f}")# added
    rscores.append(np.mean(scores))
  mean_pref_acc = np.mean(rscores)
  std_pref_acc = np.std(rscores)
  print("================================================")               # added
  print(f"FINAL SCORE = {mean_pref_acc:.4f} STD = {std_pref_acc:.4f}")    # added
  return mean_pref_acc,std_pref_acc

In [3]:
def create_encoder_model(vocab_size, embed_dim, enc_units):
    # Input of arbitrary length for infix sequence
    enc_inputs = Input(shape=(None,), name="enc_inputs")  # (batch, variable_length)
    # Embedding
    x = Embedding(input_dim=vocab_size,
                  output_dim=embed_dim,
                  name="enc_embedding")(enc_inputs)
    # LSTM returns all hidden states and the final (h, c) states; 
    enc_outputs, state_h, state_c = LSTM(enc_units,
                                        return_sequences=True,
                                        return_state=True,
                                        name="encoder_lstm")(x)
    # Model outputs encoder hidden sequence and final states
    return Model(enc_inputs, [enc_outputs, state_h, state_c], name="encoder_model")

def create_decoder_model(vocab_size, embed_dim, dec_units):
    # Decoder inputs (token IDs) of variable length for teacher forcing
    dec_inputs = Input(shape=(None,), name="dec_inputs")  # (batch, variable_length)
    # Encoder outputs and states come from encoder
    enc_outputs = Input(shape=(None, dec_units), name="enc_outputs")  # (batch, enc_len, units)
    state_h_in  = Input(shape=(dec_units,), name="h_in")
    state_c_in  = Input(shape=(dec_units,), name="c_in")

    # Embedding of decoder input
    x = Embedding(input_dim=vocab_size,
                  output_dim=embed_dim,
                  name="dec_embedding")(dec_inputs)
    # Decoder LSTM consumes masked embeddings and initial states
    dec_outputs, state_h_out, state_c_out = LSTM(dec_units,
                                                return_sequences=True,
                                                return_state=True,
                                                name="decoder_lstm")(x,
                                                                   initial_state=[state_h_in, state_c_in])
    # It aligns each decoder timestep to relevant encoder hidden states
    context = AdditiveAttention(name="attention")([dec_outputs, enc_outputs])
    # Concatenate context vector with decoder outputs for richer representation
    concat = Concatenate(axis=-1, name="concat")([dec_outputs, context])

    # Final projection to vocabulary distribution (softmax)
    logits = Dense(vocab_size, activation="softmax", name="vocab_dist")(concat)

    # Return logits and new states for inference
    return Model(
        [dec_inputs, enc_outputs, state_h_in, state_c_in],
        [logits, state_h_out, state_c_out],
        name="decoder_model"
    )

# Combined Seq2Seq Model: for training with teacher forcing
def create_seq2seq_model(encoder, decoder):
    # Encoder input and decoder input
    enc_in = encoder.input       # (batch, variable_length)
    dec_in = decoder.input[0]    # (batch, variable_length)

    # Forward pass through encoder
    enc_outs, h, c = encoder(enc_in)
    # Forward pass through decoder using teacher-forcing inputs and encoder outputs/states
    logits, _, _ = decoder([dec_in, enc_outs, h, c])

    # Full model maps [enc_in, dec_in] -> logits
    return Model([enc_in, dec_in], logits, name="seq2seq_model")

# test 1 (2.8M)

In [4]:
EMBED_DIM  = 32
ENC_UNITS = 256
DEC_UNITS = 256

encoder = create_encoder_model(VOCAB_SIZE, EMBED_DIM, ENC_UNITS)
decoder = create_decoder_model(VOCAB_SIZE, EMBED_DIM, DEC_UNITS)
seq2seq = create_seq2seq_model(encoder, decoder)
PATH = "/home/pp26/ml/progettoDL/report/model/model_finale_1.8M/seq2seq_attention_model_v1.check_16.weights.h5"

seq2seq.load_weights(PATH)

W0000 00:00:1749735241.127762 1055125 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
for _ in range(20):
    print("Testing autoregressive decoding with 20 samples...")
    mean_pref_acc, std_pref_acc = test(verbose_n=0, verbose_round=False)

Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 0.9996 STD = 0.0012
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 

# test 2 (500m)

In [6]:
EMBED_DIM  = 32
ENC_UNITS = 128
DEC_UNITS = 128

encoder = create_encoder_model(VOCAB_SIZE, EMBED_DIM, ENC_UNITS)
decoder = create_decoder_model(VOCAB_SIZE, EMBED_DIM, DEC_UNITS)
seq2seq = create_seq2seq_model(encoder, decoder)
PATH = "/home/pp26/ml/progettoDL/report/model/model_finale_500m/seq2seq_attention_model_v2.1.weights.h5"

seq2seq.load_weights(PATH)

In [7]:
for _ in range(20):
    print("Testing autoregressive decoding with 20 samples...")
    mean_pref_acc, std_pref_acc = test(verbose_n=0, verbose_round=False)

Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 

# test 3  (132m)

In [10]:
# test 1
EMBED_DIM  = 16 #<-------------------------------
ENC_UNITS = 64
DEC_UNITS = 64

encoder = create_encoder_model(VOCAB_SIZE, EMBED_DIM, ENC_UNITS)
decoder = create_decoder_model(VOCAB_SIZE, EMBED_DIM, DEC_UNITS)
seq2seq = create_seq2seq_model(encoder, decoder)
PATH = "/home/pp26/ml/progettoDL/report/model/model_v4.0/seq2seq_attention_model_v4.0.weights.h5"

seq2seq.load_weights(PATH)

In [11]:
for _ in range(20):
    print("Testing autoregressive decoding with 20 samples...")
    mean_pref_acc, std_pref_acc = test(verbose_n=0, verbose_round=False)

Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 

# test 4 (42m)

In [12]:
# test 1
EMBED_DIM  = 16 #<-------------------------------
ENC_UNITS = 32
DEC_UNITS = 32

encoder = create_encoder_model(VOCAB_SIZE, EMBED_DIM, ENC_UNITS)
decoder = create_decoder_model(VOCAB_SIZE, EMBED_DIM, DEC_UNITS)
seq2seq = create_seq2seq_model(encoder, decoder)
PATH = "/home/pp26/ml/progettoDL/report/model/model_v4.1/seq2seq_attention_model_v4.1.weights.h5"

seq2seq.load_weights(PATH)

In [13]:
for _ in range(20):
    print("Testing autoregressive decoding with 20 samples...")
    mean_pref_acc, std_pref_acc = test(verbose_n=0, verbose_round=False)

Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 1.0000 STD = 0.0000
Testing autoregressive decoding with 20 samples...
FINAL SCORE = 